In [22]:
# import torch
# import numpy as np
# import random

# def set_seed(seed=42):
#     # 1. Set the seed for PyTorch's CPU and GPU RNGs
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
    
#     # 2. Set the seed for Python's RNG
#     random.seed(seed)
    
#     # 3. Set the seed for NumPy's RNG
#     np.random.seed(seed)
    
#     # 4. (Optional) Disable non-deterministic algorithms for CUDA
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seed(42)


# Create Dataset

In [23]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="/root/autodl-tmp/generate_dataset/mix")
dataset

Resolving data files:   0%|          | 0/600 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/300 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 600
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 300
    })
})

In [24]:
from classification.my_dataset import get_cat_vs_dog

# train_dataset, val_dataset, test_dataset, num_of_category = get_cat_vs_dog()
train_dataset = dataset['train']
test_dataset = dataset['test']

max_epochs = 16
num_of_category = 2

In [25]:
from classification.net import res18_preprocess


def transform(examples):
    new_img_list = []
    for item in examples["image"]:
        if item.mode in ["L", "CMYK"]:
            item = item.convert("RGB")
        new_img_list.append(res18_preprocess(item))
    examples["image"] = new_img_list
    return examples


train_dataset.set_transform(transform)
# val_dataset.set_transform(transform)
test_dataset.set_transform(transform)

# Create DataLoader

In [26]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=3)
# val_dataloader = DataLoader(val_dataset, batch_size=64, num_workers=3)
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=3)

# Set Environment

In [27]:
import torch
import os
import wandb
from pytorch_lightning.loggers import WandbLogger

torch.set_float32_matmul_precision("high")

os.environ["https_proxy"] = "http://172.20.0.113:12798"
os.environ["http_proxy"] = "http://172.20.0.113:12798"
os.environ["WANDB_NOTEBOOK_NAME"] = "classification"

# wandb.login()
# a21ecfe3d3c08ddc3e7f628f6005f63c3048cb0a

# wandb_logger = WandbLogger()

# Train the Model

In [28]:
from classification.net import Res18
from classification.lightning_model import Classifier
import lightning.pytorch as pl


model = Classifier(Res18, num_of_category)

trainer = pl.Trainer(max_epochs=max_epochs)
trainer.fit(
    model=model, train_dataloaders=train_dataloader
)

trainer.test(model=model, dataloaders=test_dataloader)

wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | net            | Res18              | 11.2 M
1 | train_accuracy | MulticlassAccuracy | 0     
2 | val_accuracy   | MulticlassAccuracy | 0     
3 | test_accuracy  | MulticlassAccuracy | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


/root/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:281: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=16` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9600000381469727     │
│         test_loss         │    0.16725598275661469    │
└───────────────────────────┴───────────────────────────┘